# Kubernetes

In [1]:
#!pip install ipynb
from ipynb.fs.full.facturxWidget import *

from ipynb.fs.full.kubernetesTool import *

from ipynb.fs.full.invoiceSmartcontract import *

from ipynb.fs.full.blockchain import *

aubonmoulin-79666dbdd9-mn8bf
austriafactor-7fbf86dcf-25qkt
cc-deployer-746f97765-dmjnw
hl-fabric-data-share-deployment-6756f45c89-sbnrc
invoice-chaincode-external-6d6dcf7d47-45j75
invoice-net-chaincode-external-54cbcd776-8b7m6
lyon-impots-77c667fb49-v8krj
megustaolive-f477b7b59-ql8nz
orderer-org1-scray-org-6dd8878cf5-725gs
peer-11233-bd7b47d88-4kqcf
peer-12188-595bf9fcd4-xhvft
peer-1285-656bd6d976-hmv7g
peer-17802-6cddbc865b-6cj6f
peer-17975-75668f595d-pmvlj
peer-18614-c4c75768d-qb6vd
peer-27349-6f959c996b-k8c6h
peer-27953-7468b8f67b-sgdmj
peer-31954-7c6f95b9dd-jmhhl
peer-3786-6df45f5d96-sz9ks
peer-5557-b5f44cfcb-w8z6c
peer-8373-794c47f854-mll4j
peer-9603-78bcf474cb-x49q2
peer205-599cbd5dd7-lw94k
peer300-dc4859bfb-94f8s
peer401-75f46f8d84-2ltsj
peer404-799f74f9c4-727lz
peer405-788f7d9c48-7wdkc
peer505-64f744d5d-hmjwm
peer506-67d98d58dd-rrc5z
peer508-69d9b48964-z8tmv
peer600-79df66bfbb-b4lh8
peer601-84888f67bd-8h56p
peer602-d8577b45d-l2wsq
peer612-5c6fb597d4-jdd74
peer660-c975d5c66-fzf5

Error from server (NotFound): services "olya0" not found
Error from server (NotFound): services "olya0" not found


In [2]:
_blockchain = Blockchain(channel_name='invoicing25',shared_fs='10.15.130.111:80',sharepoint=SharePoint())
cc = Chaincode()

In [3]:
_blockchain.addPeer('aubonmoulin', hostname='fr',ca_country='FR',ca_province='Provence-Alpes-Cote d\'Azur',ca_locality='Malaucene')
_blockchain.addPeer('lyon-impots', hostname='gouv.fr',ca_country='FR',ca_province='Auvergne-Rhone-Alpes',ca_locality='Lyon')
_blockchain.addPeer('austriafactor', hostname='at',ca_country='AT',ca_province='Steiermark',ca_locality='Graz')
_blockchain.addPeer('megustaolive', hostname='es',ca_country='ES',ca_province='Andalusia',ca_locality='Dos Hermanas')

### user

In [27]:
_blockchain.installWalletTool()

In [13]:
_user = User(peer=_blockchain.peers['aubonmoulin'], name='Tony.Dubois@aubonmoulin.fr', channel_name=_blockchain.channel_name)
#User(peer=_blockchain.peers['aubonmoulin'], name='Alexandre.Payet@majolieboutique.net', channel_name=_blockchain.channel_name)

In [18]:
_user.smart_contract.getAllRoles()

([{'name': 'x509::CN=Auguste.Boucher@lyon-impots.gouv.fr,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.lyon-impots.gouv.fr,O=lyon-impots.gouv.fr,L=Lyon,ST=Auvergne-Rhone-Alpes,C=FR',
   'roles': ['TaxInspector'],
   'taxInspector': ''},
  {'name': 'x509::CN=Emma.Steiner@austriafactor.at,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.austriafactor.at,O=austriafactor.at,L=Graz,ST=Steiermark,C=AT',
   'roles': ['Factor'],
   'taxInspector': 'x509::CN=Auguste.Boucher@lyon-impots.gouv.fr,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.lyon-impots.gouv.fr,O=lyon-impots.gouv.fr,L=Lyon,ST=Auvergne-Rhone-Alpes,C=FR'},
  {'name': 'x509::CN=Pedro.Sanchez@megustaolive.es,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.megustaolive.es,O=megustaolive.es,L=Dos,ST=Andalusia,C=ES',
   'roles': ['Seller', 'Buyer'],
   'taxInspector': 'x509::CN=Auguste.Boucher@lyon-impots.gouv.fr,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.lyon-impots.gouv.fr,O=lyon-impots.gouv.f

In [28]:
for key,_peer in _blockchain.peers.items():
    _organizations = _peer.wallet_tool.getPeerOrganizations()
    for _organization in _organizations:
        porg = Organization(name=_organization)
        _users = _peer.wallet_tool.getPeerUsers(_organization)
        for _user in _users:
            puser = User(peer=_peer, name=_user, channel_name=_blockchain.channel_name)
            porg.addUser(user=puser)
        _peer.organizations[_organization]=porg
    

In [ ]:
_blockchain.peers['aubonmoulin'].organizations['aubonmoulin.fr'].users['Tony.Dubois@aubonmoulin.fr'].__dict__

#_blockchain.peers['lyon-impots'].organizations
#_blockchain.peers

In [ ]:
_smart_contract_aubonmoulin = InvoiceSmartContract(peer=_blockchain.peers['aubonmoulin'], user='Tony.Dubois@aubonmoulin.fr', channel_name=_blockchain.channel_name)

In [ ]:
_smart_contract_lyon_impots = InvoiceSmartContract(peer=_blockchain.peers['lyon-impots'], user='Auguste.Boucher@lyon-impots.gouv.fr', channel_name=_blockchain.channel_name)

In [ ]:
_smart_contract.getSubmittingClientIdentity()
_smart_contract.getAllRoles()

In [ ]:
_blockchain.peers['aubonmoulin'].wallet_tool.addUser('Tony.Dubois','aubonmoulin.fr','aubonmoulin.fr')
_blockchain.peers['aubonmoulin'].wallet_tool.addUser('Stephanie.Hoarau','hotelsaintdenis.re','aubonmoulin.fr')
_blockchain.peers['aubonmoulin'].wallet_tool.addUser('Alexandre.Payet','majolieboutique.net','aubonmoulin.fr')
_blockchain.peers['austriafactor'].wallet_tool.addUser('Emma.Steiner','austriafactor.at','austriafactor.at')
_blockchain.peers['megustaolive'].wallet_tool.addUser('Pedro.Sanchez','megustaolive.es','megustaolive.es')
_blockchain.peers['lyon-impots'].wallet_tool.addUser('Auguste.Boucher','lyon-impots.gouv.fr','lyon-impots.gouv.fr')

In [ ]:
def appendRoles(peers):
    peer=peers['lyon-impots']
    appendRole(peer,user='Auguste.Boucher@lyon-impots.gouv.fr',role='TaxInspector')
    peer_taxinspector=peer
    #taxinspector=get_person_names_with_role('TaxInspector',roles)[0].split(',')[0].split('=')[1]  
    taxinspector='Auguste.Boucher@lyon-impots.gouv.fr'

    identity_taxinspector = getSubmittingClientIdentity(peer_taxinspector, user=taxinspector)[0]

    peer=peers['aubonmoulin']
    appendRole(peer,user='Tony.Dubois@aubonmoulin.fr',role='Seller',taxInspector=identity_taxinspector)
    #appendRole(peer,user='Stephanie.Hoarau',role='Buyer')
    #appendRole(peer,user='Alexandre.Payet',role='Buyer')

    peer=peers['austriafactor']
    appendRole(peer,user='Emma.Steiner@austriafactor.at',role='Factor',taxInspector=identity_taxinspector)

    peer=peers['megustaolive']
    appendRole(peer,user='Pedro.Sanchez@megustaolive.es',role='Buyer',taxInspector=identity_taxinspector)
    appendRole(peer,user='Pedro.Sanchez@megustaolive.es',role='Seller',taxInspector=identity_taxinspector)

In [ ]:
def getUser(_blockchain, _user):
    _org = _user.split('@')[1]
    for key,peer in _blockchain.peers.items():
        if _org in peer.organizations.keys():
            if _user in peer.organizations[_org].users.keys():
                return peer.organizations[_org].users[_user]

In [ ]:
getUser(_blockchain,'Auguste.Boucher@lyon-impots.gouv.fr').smart_contract.appendRole(role='TaxInspector')
identity_taxinspector = getUser(_blockchain, 'Auguste.Boucher@lyon-impots.gouv.fr').smart_contract.getSubmittingClientIdentity()[0]
getUser(_blockchain,'Emma.Steiner@austriafactor.at').smart_contract.appendRole(role='Factor',taxInspector=identity_taxinspector)
getUser(_blockchain,'Tony.Dubois@aubonmoulin.fr').smart_contract.appendRole(role='Seller',taxInspector=identity_taxinspector)
getUser(_blockchain,'Pedro.Sanchez@megustaolive.es').smart_contract.appendRole(role='Seller',taxInspector=identity_taxinspector)
getUser(_blockchain,'Pedro.Sanchez@megustaolive.es').smart_contract.appendRole(role='Buyer',taxInspector=identity_taxinspector)

In [ ]:
getUser(_blockchain,'Tony.Dubois@aubonmoulin.fr').smart_contract.getAllRoles()

In [ ]:
#_smart_contract_lyon_impots.getAllRoles()
_smart_contract_aubonmoulin.getRoles()
#_out
_smart_contract_lyon_impots.getRoles()
_smart_contract_lyon_impots.getAllKeys()

In [ ]:
_smart_contract_lyon_impots.getAllKeys()

## cleanup

In [ ]:
_blockchain.deletePeers()
#_blockchain.peers

## add channel

In [ ]:
_blockchain.add_channel('invoicing25')

## debug

In [ ]:
out2.outputs[1]['text'].splitlines()

In [ ]:
_out.outputs[0]['text'].splitlines()

## create peers

### endorse and join

In [ ]:
## endorse and join
import ipywidgets as widgets

#orderer_ip,orderer_hostname,orderer_port,channel_name,shared_fs_host

In [ ]:
_blockchain.endorse_and_join_all_peers()

## upload chaincode to sharepoint

In [ ]:
strlist = ['/bin/sh /opt/create-archive.sh',cc.hostname,cc.port,cc.label,cc.shared_fs]
cmd = toCmd(strlist)
print(cmd)
#cmd = export_cmd + '  && ' + cmd
output = executePeerCmd(getPod('cc-deployer'), cmd,cli='cc-deployer')

## install chaincode

In [ ]:
_blockchain.installAndAproveChaincode(channel_name=_blockchain.channel_name, chain_code=cc)

In [ ]:
_blockchain.peers['aubonmoulin'].commitChaincode(_blockchain.channel_name,pkgid=cc.pkgid)

## add roles

In [45]:
deleteAllInvoices(User(peer=_blockchain.peers['aubonmoulin'], name='Tony.Dubois@aubonmoulin.fr', channel_name=_blockchain.channel_name).smart_contract)              

{"function":"DeleteAsset","Args":["asset14"]}
{"function":"DeleteAsset","Args":["asset17"]}
{"function":"DeleteAsset","Args":["asset3"]}
{"function":"DeleteAsset","Args":["asset6"]}


In [55]:
from typing import Dict, Any
import hashlib
import json

def dict_hash(dictionary: Dict[str, Any]) -> str:
    """MD5 hash of a dictionary."""
    dhash = hashlib.md5()
    # We need to sort arguments so {'a': 1, 'b': 2} is
    # the same as {'b': 2, 'a': 1}
    encoded = json.dumps(dictionary, sort_keys=True).encode()
    dhash.update(encoded)
    return dhash.hexdigest()


In [46]:
createInvoices(invoices,_blockchain)            

'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
{"function":"CreateInvoice","Args":["asset3","x509::CN=Tony.Dubois@aubonmoulin.fr,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.aubonmoulin.fr,O=aubonmoulin.fr,L=Malaucene,ST=Provence-Alpes-Cote,C=FR","x509::CN=Alexandre.Payet@majolieboutique.net,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.aubonmoulin.fr,O=aubonmoulin.fr,L=Malaucene,ST=Provence-Alpes-Cote,C=FR","-6679375540195564355","AV-2017-0005","14.99","218.48","FR","FR","","False","False","False","False","","","False"]}
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
{"function":"CreateInvoice","Args":["asset6","x509::CN=Tony.Dubois@aubonmoulin.fr,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.aubonmoulin.fr,O=aubonmoulin.fr,L=Malaucene,ST=Provence-Alpes-Cote,C=FR","x509::CN=Alexandre.Payet@majolieboutique.net,OU=admin,O=kubernetes.research.dev.seeburger.de::

In [34]:
User(peer=_blockchain.peers['aubonmoulin'], name='Tony.Dubois@aubonmoulin.fr', channel_name=_blockchain.channel_name).smart_contract.listInvoices()

('', Output())

In [26]:
invoice=invoices.iloc[3]
getUser(_blockchain,get_CN_of_invoice(invoice,'sellerTradeParty'))

'Tony.Dubois@aubonmoulin.fr'

In [ ]:
seller.smart_contract.getAllKeys()[0]

In [ ]:
invoices = get_invoices()
#createInvoices(invoices,_blockchain.peers)
#invoices

In [ ]:
def get_org_of_invoice(invoice,party):
    _name=strip_accents(getPathValue(invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/' + party + '/definedTradeContact/')[0]['personName']['value'].replace(' ','.'))
    _email=getPathValue(invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/' + party + '/definedTradeContact/')[0]['emailURIUniversalCommunication']['uriid']['value']
    return _email.split('@')[1]

In [ ]:
User(peer=_blockchain.peers['aubonmoulin'], user='Tony.Dubois@aubonmoulin.fr', channel_name=_blockchain.channel_name).smart_contract.getSubmittingClientIdentity()

In [ ]:
get_CN_of_invoice(invoice,'sellerTradeParty')

In [11]:
#list(asset)
_smart_contract_aubonmoulin.getEmptyInvoice()[0]

NameError: name '_smart_contract_aubonmoulin' is not defined

In [ ]:
InvoiceSmartContract(peer=_blockchain.peers['lyon-impots'], user='Auguste.Boucher@lyon-impots.gouv.fr').appendRole(role='TaxInspector')

In [ ]:
InvoiceSmartContract(peer=_blockchain.peers['aubonmoulin'], user='Tony.Dubois@aubonmoulin.fr').appendRole(role='Seller', taxInspector='')

In [ ]:
strlist = ['/bin/ls',  '/mnt/conf/organizations/peerOrganizations/']
peer=_blockchain.peers['aubonmoulin']
executePeerCmd(peer.pod_name,toCmd(strlist)).splitlines() 

In [ ]:
appendRoles(_blockchain.peers)
#peer=_blockchain.peers['lyon-impots']
#appendRole(peer,user='Auguste.Boucher@lyon-impots.gouv.fr',role='TaxInspector')

In [7]:
peer=_blockchain.peers['lyon-impots']
getRoles(peer,user='Auguste.Boucher@lyon-impots.gouv.fr')[0]

'Command \'[\'/home/jovyan/work/usr/bin/kubectl\', \'exec\', \'-t\', \'lyon-impots-77c667fb49-v8krj\', \'-c\', \'scray-peer-cli\', \'--\', \'/bin/sh\', \'-c\', \'export CORE_PEER_MSPCONFIGPATH=/mnt/conf/organizations/peerOrganizations/lyon-impots.gouv.fr/users/Auguste.Boucher@lyon-impots.gouv.fr/msp/  && peer chaincode query -C invoicing18 -n basic -c \\\'{"function":"GetRoles","Args":[]}\\\'\']\' returned non-zero exit status 1.'

In [ ]:
callopt='{"function":"AppendRole","Args":["x509::CN=Auguste.Boucher@lyon-impots.gouv.fr,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.lyon-impots.gouv.fr,O=lyon-impots.gouv.fr,L=Lyon,ST=Auvergne-Rhone-Alpes,C=FR","TaxInspector",""]}'

In [10]:
peer=_blockchain.peers['lyon-impots']
getAllKeys(peer)[1]

Output()

In [ ]:
peer=_blockchain.peers['lyon-impots']
callopt= '{"function":"GetAllKeys","Args":[' + ']}'
#peer.query(callopt,channel_name='invoicing18',user='Auguste.Boucher@lyon-impots.gouv.fr')
peer.query(callopt,channel_name='invoicing18',user='User1')

In [ ]:
getAllRoles(peer)
getRoles(peer, user='Auguste.Boucher@lyon-impots.gouv.fr')

In [ ]:
getRoleTransactions(peer)

In [ ]:
invoices = get_invoices()
createInvoices(invoices,_blockchain.peers)
#invoices

In [ ]:
peer=_blockchain.peers['aubonmoulin']
getPeerOrganizations(peer)

In [ ]:
get_peer_of_user('Tony.Dubois@aubonmoulin.fr',_blockchain.peers)

In [ ]:
peer=_blockchain.peers['aubonmoulin']
listInvoices(peer, user='Tony.Dubois@aubonmoulin.fr')[1]

In [ ]:
peer=_blockchain.peers['aubonmoulin']
InvoiceSmartContract(peer=peer, user='Tony.Dubois@aubonmoulin.fr').listInvoices()[0]

In [ ]:
InvoiceSmartContract(peer=peer, user='Tony.Dubois@aubonmoulin.fr').__dict__

In [ ]:
deleteAllInvoices(peer)  